In [12]:
import pandas as pd
import numpy as np
import math
import datetime

In [13]:
train_m = pd.read_csv('train_starting.csv', index_col=0)
test_m = pd.read_csv('test_starting.csv', index_col = 0)

In [14]:
#original column names of training data
print list(train_m.columns)

['id', 'date_account_created', 'timestamp_first_active', 'date_first_booking', 'gender', 'age', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser', 'country_destination', 'lag_account_created', 'lag_first_active', 'lag_account_created_first_active', 'bookings', 'population_in_thousands', 'sum_secs_elapsed', 'counts']


In [15]:
#the toremove is a list of columns to be removed from the original dataset
excl = list(train_m.columns)
toremove = ['id', 'date_account_created', 'timestamp_first_active', 'date_first_booking', 'population_in_thousands',
            'lag_account_created', 'bookings', 'lag_first_active', 
            #'lag_account_created_first_active', 
              'country_destination'] 
map(lambda x: excl.remove(x), toremove)

[None, None, None, None, None, None, None, None, None]

In [16]:
##applying to remove to the dataset and left with a clean training and test dataset
train_xcl = train_m.loc[:, excl]
test_xcl = test_m.loc[:, excl]

In [17]:
#convert training and test x data into a sparce matrix dictionary
from sklearn.feature_extraction import DictVectorizer

frames = [train_xcl, test_xcl]
all_data = pd.concat(frames)
all_data = all_data.T.to_dict().values()
train_xdic = train_xcl.T.to_dict().values()
test_xdic = test_xcl.T.to_dict().values()

vec = DictVectorizer()
vec.fit(all_data)
train_xvec = vec.transform(train_xdic)
test_xvec = vec.transform(test_xdic)

/usr/local/lib/python2.7/site-packages/pandas/core/frame.py:840: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  "columns will be omitted.", UserWarning)


In [18]:
train_y = train_m.loc[:,'country_destination']

In [19]:
#dumbify y variable
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train_y)
train_ytrans = le.transform(train_y)
print type(train_ytrans)

<type 'numpy.ndarray'>


In [20]:
#First model fitted using XGBoost
import xgboost as xgb
import numpy as np
params = {}
params["objective"] = "multi:softmax"
params["num_class"] = 12
params["eta"] = 0.3
params["min_child_weight"] = 6
params["subsample"] = 0.7
params["colsample_bytree"] = 0.7
params["scale_pos_weight"] = 1
params["max_depth"] = 3
params['eval_metric'] = "ndcg@5"
params['nthread'] = 4

plst = list(params.items())

dtrain = xgb.DMatrix(train_xvec, label=train_ytrans)
dtest = xgb.DMatrix(test_xvec)
num_round = 25

In [21]:
#fitting first model
model = xgb.train(plst, dtrain, num_round, verbose_eval=True)

In [22]:
#This model2 was fit using xgboost.XGBClassifier, which should be treated as any ohter model in scikitlearn
import numpy as np
model2 = xgb.XGBClassifier(max_depth = 5, learning_rate = 0.1, n_estimators = 30, silent = False, \
                           objective ="multi:softprob")

In [23]:
#fitting model two
model2.fit(train_xvec, train_ytrans, verbose = True, eval_metric = "ndcg@5")

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=30, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=False, subsample=1)

In [24]:
#prediction of first model 
pred = map(int,model.predict(dtrain))
predinv = le.inverse_transform(pred)

In [25]:
#prediction of second model
pred2 = model2.predict(train_xvec)
pred2inv = le.inverse_transform(pred2)

In [36]:
dtrain_x